<a href="https://colab.research.google.com/github/JuanMBuitrago/CryptoEda/blob/main/Crypto_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime

In [ ]:
lista_crypto = ['bitcoin', 'ethereum', 'dogecoin', 'ripple', 'cardano', 'solana', 'tron', 'polkadot', 'litecoin', 'avalanche-2']
tickers = ['BTC', 'ETH', 'DOGE', 'XRP', 'ADA', 'SOL', 'TRX', 'DOT', 'LTC', 'AVAX']
lista_crypto_s = ['bitcoin', 'ethereum']


In [ ]:
def df_gecko(crypto, ticker=None):
  if ticker is None:
      ticker = crypto
  url = "https://api.coingecko.com/api/v3/coins/" + crypto + "/market_chart/range?vs_currency=usd&from=1514764800&to=1672473599"
  respuesta = requests.get(url)
  if respuesta.status_code == 200:
    data = pd.DataFrame(respuesta.json())
    data['day'] = pd.to_datetime(data['prices'].apply(lambda x: x[0]), unit='ms').dt.strftime('%d-%m-%Y')
    data['market_cap_' + ticker] = data['market_caps'].apply(lambda x:x[1])
    data['price_' + ticker] = data['prices'].apply(lambda x:x[1])
    data['volume_' + ticker] = data['total_volumes'].apply(lambda x:x[1])
    data.drop(columns= ['market_caps', 'prices', 'total_volumes'], inplace= True)
    return data
  else:
    print(f"La solicitud a la API para {crypto} falló dando un código de estado: {respuesta.status_code}")


In [ ]:
df_combinado = df_gecko(lista_crypto[0], tickers[0])
for i in range(1, len(lista_crypto)):
  print(lista_crypto[i])
  temp_df = df_gecko(lista_crypto[i], tickers[i])
  df_combinado = df_combinado.merge(temp_df, on='day', how='outer')

ethereum
dogecoin
ripple
cardano
solana
tron
polkadot
litecoin
avalanche-2


In [ ]:
df_combinado.tail()

In [ ]:
api_url = 'https://api.alternative.me/fng/?limit=0'
respuesta = requests.get(api_url)

if respuesta.status_code == 200:
    fear_data = respuesta.text
    fear_data = json.loads(fear_data)
else:
    print(f"La solicitud a la API de 'Crypto Fear & Greed Index' falló dando un código de estado: {respuesta.status_code}")
df_fear = pd.DataFrame(fear_data['data'])
df_fear['day'] = df_fear['timestamp'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%d-%m-%Y'))
df_fear.drop(columns=['timestamp', 'time_until_update'], inplace=True)
df_fear.rename(columns={'value': 'fear_value', 'value_classification': 'fear_classification'}, inplace=True)

In [ ]:
df_combinado = df_combinado.merge(df_fear, on='day', how='outer')
df_combinado['day'] = pd.to_datetime(df_combinado['day'], format='%d-%m-%Y')
fecha_tope = datetime.strptime('31-12-2022', '%d-%m-%Y')
mascara_fecha = df_combinado['day'] <= fecha_tope
df_combinado = df_combinado[mascara_fecha]

In [ ]:
df_combinado.tail()

,day,market_cap_BTC,price_BTC,volume_BTC,market_cap_ETH,price_ETH,volume_ETH,market_cap_DOGE,price_DOGE,volume_DOGE,...,price_DOT,volume_DOT,market_cap_LTC,price_LTC,volume_LTC,market_cap_AVAX,price_AVAX,volume_AVAX,fear_value,fear_classification
1821,2022-12-27,3.250369e+11,16900.081872,1.257500e+10,1.476973e+11,1226.253415,3.071222e+09,1.039291e+10,0.075731,3.316964e+08,...,4.570317,1.001199e+08,5.062685e+09,70.518679,4.937022e+08,3.648145e+09,11.699718,9.961573e+07,27,Fear
1822,2022-12-28,3.213515e+11,16701.840211,1.743606e+10,1.460305e+11,1211.819562,4.221451e+09,1.012104e+10,0.073680,3.710858e+08,...,4.478367,1.183746e+08,4.941955e+09,68.722001,4.653066e+08,3.627298e+09,11.647524,1.097341e+08,28,Fear
1823,2022-12-29,3.182889e+11,16538.906094,1.911361e+10,1.432418e+11,1188.728664,5.177421e+09,9.666874e+09,0.070388,6.424461e+08,...,4.280846,1.788469e+08,4.766597e+09,66.276459,4.809904e+08,3.489305e+09,11.197758,1.152648e+08,28,Fear
1824,2022-12-30,3.202973e+11,16644.379087,1.568373e+10,1.448313e+11,1201.543106,4.174716e+09,9.782541e+09,0.071101,3.684301e+08,...,4.307811,1.062141e+08,4.801367e+09,66.732654,3.549561e+08,3.424420e+09,10.989320,1.096235e+08,28,Fear
1825,2022-12-31,3.197220e+11,16604.020520,1.749733e+10,1.446115e+11,1199.110497,4.110733e+09,9.407880e+09,0.068406,5.449513e+08,...,4.324199,9.901919e+07,4.892375e+09,67.940936,3.485045e+08,3.389684e+09,10.870447,1.240163e+08,25,Extreme Fear


In [ ]:
df_combinado.columns

Index(['day', 'market_cap_BTC', 'price_BTC', 'volume_BTC', 'market_cap_ETH',
       'price_ETH', 'volume_ETH', 'market_cap_DOGE', 'price_DOGE',
       'volume_DOGE', 'market_cap_XRP', 'price_XRP', 'volume_XRP',
       'market_cap_ADA', 'price_ADA', 'volume_ADA', 'market_cap_SOL',
       'price_SOL', 'volume_SOL', 'market_cap_TRX', 'price_TRX', 'volume_TRX',
       'market_cap_DOT', 'price_DOT', 'volume_DOT', 'market_cap_LTC',
       'price_LTC', 'volume_LTC', 'market_cap_AVAX', 'price_AVAX',
       'volume_AVAX', 'fear_value', 'fear_classification'],
      dtype='object')

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np  # Import numpy for handling NaN values

# color_map = {
#     'Extreme Fear': 'red',
#     'Fear': 'orange',
#     'Neutral': 'gray',
#     'Greed': 'green',
#     'Extreme Greed': 'darkgreen'
# }

# def map_classification_to_color(classification):
#     return color_map.get(classification, 'black')  # Default color for NaN

# colors = [map_classification_to_color(fear_classification) for fear_classification in df_combinado['fear_classification']]

# plt.figure(figsize=(10, 6))
# plt.scatter(df_combinado['day'], df_combinado['price_BTC'], c=colors, label=df_combinado['fear_classification'])
# plt.xlabel('Day')
# plt.ylabel('BTC Price')
# plt.title('Correlation between Fear and Greed Indices')
# plt.legend()

# plt.show()

In [ ]:
df_combinado

,day,market_cap_BTC,price_BTC,volume_BTC,market_cap_ETH,price_ETH,volume_ETH,market_cap_DOGE,price_DOGE,volume_DOGE,...,price_DOT,volume_DOT,market_cap_LTC,price_LTC,volume_LTC,market_cap_AVAX,price_AVAX,volume_AVAX,fear_value,fear_classification
0,2018-01-01,2.364403e+11,14093.606831,2.345479e+09,7.505663e+10,776.080330,1.101628e+09,1.023601e+09,0.009091,3.857289e+07,...,NaN,NaN,1.257716e+10,230.462120,3.654324e+08,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,2.570796e+11,15321.932852,4.130218e+09,8.482222e+10,876.890259,2.720973e+09,1.051233e+09,0.009335,6.782355e+07,...,NaN,NaN,1.392244e+10,255.048185,8.125302e+08,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,2.615052e+11,15583.885538,3.310356e+09,9.317853e+10,963.055751,2.308657e+09,1.080254e+09,0.009592,5.158160e+07,...,NaN,NaN,1.354357e+10,248.042194,6.184902e+08,NaN,NaN,NaN,NaN,NaN
3,2018-01-04,2.681247e+11,15976.365194,4.217387e+09,9.468115e+10,978.416104,2.756635e+09,1.137418e+09,0.010098,8.583633e+07,...,NaN,NaN,1.337223e+10,244.834372,8.453734e+08,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,3.077740e+11,18336.922980,6.275688e+09,9.726923e+10,1004.894972,2.894789e+09,1.559193e+09,0.013841,1.574593e+08,...,NaN,NaN,1.388403e+10,254.138525,8.335260e+08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2022-12-27,3.250369e+11,16900.081872,1.257500e+10,1.476973e+11,1226.253415,3.071222e+09,1.039291e+10,0.075731,3.316964e+08,...,4.570317,1.001199e+08,5.062685e+09,70.518679,4.937022e+08,3.648145e+09,11.699718,9.961573e+07,27,Fear
1822,2022-12-28,3.213515e+11,16701.840211,1.743606e+10,1.460305e+11,1211.819562,4.221451e+09,1.012104e+10,0.073680,3.710858e+08,...,4.478367,1.183746e+08,4.941955e+09,68.722001,4.653066e+08,3.627298e+09,11.647524,1.097341e+08,28,Fear
1823,2022-12-29,3.182889e+11,16538.906094,1.911361e+10,1.432418e+11,1188.728664,5.177421e+09,9.666874e+09,0.070388,6.424461e+08,...,4.280846,1.788469e+08,4.766597e+09,66.276459,4.809904e+08,3.489305e+09,11.197758,1.152648e+08,28,Fear
1824,2022-12-30,3.202973e+11,16644.379087,1.568373e+10,1.448313e+11,1201.543106,4.174716e+09,9.782541e+09,0.071101,3.684301e+08,...,4.307811,1.062141e+08,4.801367e+09,66.732654,3.549561e+08,3.424420e+09,10.989320,1.096235e+08,28,Fear


In [ ]:
from google.colab import files

df_combinado.to_csv('df_crypto.csv', index=False)
files.download('df_crypto.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>